# 🎓 AI-Powered Interactive Learning Assistant for Classrooms

## OpenVINO Unnati Hackathon Project

This notebook demonstrates the development of an intelligent classroom companion that enhances learning experiences through AI-powered features including question answering, content summarization, multimodal interaction, and personalized learning support.

### 🎯 Project Objectives

- **Enhance Classroom Engagement**: Interactive Q&A and real-time feedback
- **Improve Accessibility**: Multimodal support (text, speech, visual)
- **Automate Content Generation**: Lesson summaries and explanations
- **Personalize Learning**: Adaptive content based on student progress
- **Optimize Performance**: OpenVINO acceleration for real-time interaction

### 🛠️ Technology Stack

- **AI Models**: LLMs, Transformers, Whisper, BLIP for multimodal capabilities
- **Optimization**: OpenVINO for Intel CPU/GPU/NPU acceleration
- **Backend**: FastAPI for robust API services
- **Frontend**: Streamlit for interactive web interface
- **Speech**: Whisper for STT, Coqui TTS for text-to-speech

---

## 1. Define the Use Case

### 🎯 Identifying Classroom Challenges

Our AI-powered learning assistant addresses several key educational challenges:

1. **Student Engagement**: Many students struggle to stay engaged during lectures
2. **Accessibility**: Students with different learning styles need varied content formats
3. **Real-time Support**: Teachers need instant help with content generation and Q&A
4. **Personalization**: One-size-fits-all approach doesn't work for diverse learners
5. **Content Summarization**: Students need quick summaries of complex lessons

### 👥 Target Users

- **Primary**: Students (K-12 and higher education)
- **Secondary**: Teachers and educators
- **Tertiary**: Educational administrators

### 📊 Success Metrics

- Response time < 2 seconds for real-time interaction
- 95%+ accuracy in question answering
- Multimodal support (text, speech, visual)
- Seamless integration with existing educational workflows

In [ ]:
# Import required libraries and set up environment
import os
import sys
import time
import warnings
from pathlib import Path
import logging

# Data handling and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Set up project paths
PROJECT_ROOT = Path("../")  # Assuming notebook is in notebooks/ directory
sys.path.append(str(PROJECT_ROOT))

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set random seed for reproducibility
np.random.seed(42)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("✅ Environment setup completed!")
print(f"📁 Project root: {PROJECT_ROOT.absolute()}")
print(f"🐍 Python version: {sys.version}")

# Display system information
import platform
print(f"💻 System: {platform.system()} {platform.release()}")
print(f"🏗️  Architecture: {platform.architecture()[0]}")

# Check if we're in the correct environment
try:
    import torch
    print(f"🔥 PyTorch version: {torch.__version__}")
except ImportError:
    print("⚠️  PyTorch not installed")

try:
    import transformers
    print(f"🤗 Transformers version: {transformers.__version__}")
except ImportError:
    print("⚠️  Transformers not installed")

## 2. Select and Download AI Models

### 🤖 Model Selection Strategy

We'll use a combination of open-source models optimized for educational tasks:

| Task | Model | Source | Rationale |
|------|--------|---------|-----------|
| **Question Answering** | DistilBERT-QA | Hugging Face | Lightweight, fast, educational-friendly |
| **Summarization** | BART-CNN | Hugging Face | Excellent for educational content |
| **Speech Recognition** | Whisper-base | OpenAI | Multilingual, robust |
| **Text-to-Speech** | Tacotron2 | Coqui TTS | Natural sounding voice |
| **Image Captioning** | BLIP | Salesforce | Multimodal understanding |

### 📦 Model Download and Setup

In [ ]:
# Model download and initialization
from transformers import (
    AutoTokenizer, AutoModelForQuestionAnswering, 
    AutoModelForSeq2SeqLM, pipeline
)

# Define model configurations
MODELS_CONFIG = {
    "question_answering": {
        "model_name": "distilbert-base-cased-distilled-squad",
        "task": "question-answering"
    },
    "summarization": {
        "model_name": "facebook/bart-large-cnn",
        "task": "summarization"
    },
    "text_generation": {
        "model_name": "microsoft/DialoGPT-medium",
        "task": "text-generation"
    }
}

def download_and_cache_models():
    """Download and cache all required models"""
    models = {}
    
    print("📥 Downloading AI models...")
    
    for task, config in MODELS_CONFIG.items():
        try:
            print(f"  🔄 Loading {task} model: {config['model_name']}")
            
            # Create pipeline for the task
            models[task] = pipeline(
                config["task"],
                model=config["model_name"],
                tokenizer=config["model_name"],
                device=-1  # Use CPU by default
            )
            
            print(f"  ✅ {task} model loaded successfully")
            
        except Exception as e:
            print(f"  ❌ Failed to load {task} model: {str(e)}")
            models[task] = None
    
    return models

# Download models (this may take a few minutes on first run)
ai_models = download_and_cache_models()

# Display model status
print("\n📊 Model Status Summary:")
for task, model in ai_models.items():
    status = "✅ Ready" if model is not None else "❌ Failed"
    print(f"  {task}: {status}")

print(f"\n🎯 Successfully loaded {sum(1 for m in ai_models.values() if m is not None)}/{len(ai_models)} models")

## 3. Install and Set Up OpenVINO

### 🚀 OpenVINO Installation

OpenVINO is Intel's toolkit for optimizing and deploying AI models on Intel hardware. It provides significant performance improvements for inference tasks.

#### Key Benefits:
- **Performance**: Up to 10x faster inference on Intel CPUs
- **Efficiency**: Reduced memory usage and power consumption  
- **Compatibility**: Supports multiple frameworks (PyTorch, TensorFlow, ONNX)
- **Hardware Optimization**: Optimized for Intel CPUs, GPUs, and NPUs

In [ ]:
# Install and set up OpenVINO
import subprocess
import sys

def install_openvino():
    """Install OpenVINO and related packages"""
    packages = [
        "openvino==2024.5.0",
        "optimum[openvino]",
        "nncf"  # Neural Network Compression Framework
    ]
    
    for package in packages:
        try:
            print(f"📦 Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ {package} installed successfully")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package}: {e}")

# Install OpenVINO (uncomment if not already installed)
# install_openvino()

# Import OpenVINO modules
try:
    import openvino as ov
    from openvino.tools import mo
    from optimum.intel import OVModelForQuestionAnswering, OVModelForSeq2SeqLM
    
    print("✅ OpenVINO imported successfully!")
    print(f"🔧 OpenVINO version: {ov.__version__}")
    
    # Initialize OpenVINO Core
    core = ov.Core()
    
    # Check available devices
    available_devices = core.available_devices
    print(f"💻 Available devices: {available_devices}")
    
    # Display device capabilities
    for device in available_devices:
        try:
            device_name = core.get_property(device, "FULL_DEVICE_NAME")
            print(f"  📱 {device}: {device_name}")
        except:
            print(f"  📱 {device}: Device information not available")
            
except ImportError as e:
    print(f"❌ OpenVINO import failed: {e}")
    print("💡 Please install OpenVINO: pip install openvino optimum[openvino]")
    core = None

# Create models directory for OpenVINO IR files
models_dir = Path("../models/openvino")
models_dir.mkdir(parents=True, exist_ok=True)
print(f"📁 OpenVINO models directory: {models_dir.absolute()}")

## 4. Convert and Optimize Models with OpenVINO

### 🔄 Model Conversion Process

OpenVINO converts models to Intermediate Representation (IR) format for optimal performance:

1. **Model Export**: Convert from original framework (PyTorch, TensorFlow)
2. **Optimization**: Apply graph optimizations and quantization
3. **Compilation**: Compile for specific hardware (CPU, GPU, NPU)

### ⚡ Performance Optimizations

- **Graph Optimization**: Removes redundant operations
- **Quantization**: Reduces precision (FP32 → FP16 → INT8)
- **Batching**: Optimizes for batch processing
- **Memory Layout**: Optimizes data layout for target hardware

In [ ]:
# Convert and optimize models with OpenVINO
def convert_model_to_openvino(model_name: str, task: str, device: str = "CPU"):
    """Convert a HuggingFace model to OpenVINO format"""
    
    if core is None:
        print("❌ OpenVINO not available")
        return None
    
    try:
        print(f"🔄 Converting {model_name} for {task}...")
        
        # Choose appropriate OpenVINO model class
        if task == "question-answering":
            ov_model = OVModelForQuestionAnswering.from_pretrained(
                model_name,
                export=True,
                device=device
            )
        elif task == "summarization":
            ov_model = OVModelForSeq2SeqLM.from_pretrained(
                model_name,
                export=True,
                device=device
            )
        else:
            print(f"⚠️  Task {task} not supported for OpenVINO conversion yet")
            return None
        
        print(f"✅ Successfully converted {model_name}")
        return ov_model
        
    except Exception as e:
        print(f"❌ Conversion failed for {model_name}: {str(e)}")
        return None

# Convert models to OpenVINO format
openvino_models = {}

if core is not None:
    print("🚀 Converting models to OpenVINO format...")
    
    # Convert Question Answering model
    qa_model_name = MODELS_CONFIG["question_answering"]["model_name"]
    openvino_models["qa"] = convert_model_to_openvino(
        qa_model_name, 
        "question-answering"
    )
    
    # Convert Summarization model
    sum_model_name = MODELS_CONFIG["summarization"]["model_name"]
    openvino_models["summarization"] = convert_model_to_openvino(
        sum_model_name,
        "summarization"
    )
    
    print(f"\n📊 OpenVINO Conversion Summary:")
    for task, model in openvino_models.items():
        status = "✅ Converted" if model is not None else "❌ Failed"
        print(f"  {task}: {status}")
else:
    print("⚠️  Skipping OpenVINO conversion - OpenVINO not available")

# Example: Manual model conversion using OpenVINO Model Optimizer
def demonstrate_manual_conversion():
    """Demonstrate manual model conversion process"""
    
    if core is None:
        return
    
    print("\n🔧 Manual Conversion Example:")
    print("This is how you would convert a PyTorch model manually:")
    
    example_code = '''
    # Step 1: Export PyTorch model to ONNX
    import torch
    from transformers import AutoModel, AutoTokenizer
    
    model = AutoModel.from_pretrained("distilbert-base-cased")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
    
    dummy_input = tokenizer("Hello world", return_tensors="pt")
    
    torch.onnx.export(
        model,
        (dummy_input["input_ids"], dummy_input["attention_mask"]),
        "model.onnx",
        input_names=["input_ids", "attention_mask"],
        output_names=["last_hidden_state"],
        dynamic_axes={
            "input_ids": {0: "batch_size", 1: "sequence"},
            "attention_mask": {0: "batch_size", 1: "sequence"},
            "last_hidden_state": {0: "batch_size", 1: "sequence"}
        }
    )
    
    # Step 2: Convert ONNX to OpenVINO IR
    from openvino.tools import mo
    
    ov_model = mo.convert_model("model.onnx")
    ov.save_model(ov_model, "model.xml")
    
    # Step 3: Load and compile OpenVINO model
    core = ov.Core()
    model = core.read_model("model.xml")
    compiled_model = core.compile_model(model, "CPU")
    '''
    
    print(example_code)

demonstrate_manual_conversion()

## 5. Benchmark Model Performance

### 📊 Performance Evaluation

We'll compare the performance of original models vs OpenVINO optimized versions across:

- **Inference Time**: Response latency for real-time interaction
- **Throughput**: Requests per second capability  
- **Memory Usage**: RAM and GPU memory consumption
- **Accuracy**: Ensuring optimization doesn't hurt model quality

### 🎯 Classroom Performance Requirements

For effective classroom use, we target:
- **Question Answering**: < 2 seconds response time
- **Summarization**: < 3 seconds for 1000 words
- **Speech Recognition**: < 1 second processing
- **Text-to-Speech**: < 2 seconds generation

In [ ]:
# Benchmark model performance
import time
import psutil
import gc
from typing import Dict, List, Callable

def benchmark_model(model, test_input, model_name: str, num_runs: int = 10) -> Dict:
    """Benchmark a model's performance"""
    
    if model is None:
        return {"error": "Model not available"}
    
    times = []
    memory_before = psutil.Process().memory_info().rss / 1024 / 1024  # MB
    
    # Warm up
    try:
        _ = model(test_input)
    except:
        pass
    
    # Run benchmarks
    for i in range(num_runs):
        gc.collect()  # Clean up memory
        
        start_time = time.time()
        try:
            result = model(test_input)
            end_time = time.time()
            times.append(end_time - start_time)
        except Exception as e:
            print(f"❌ Error in {model_name} run {i+1}: {str(e)}")
            continue
    
    memory_after = psutil.Process().memory_info().rss / 1024 / 1024  # MB
    
    if not times:
        return {"error": "All benchmark runs failed"}
    
    return {
        "model_name": model_name,
        "num_runs": len(times),
        "mean_time": np.mean(times),
        "std_time": np.std(times),
        "min_time": np.min(times),
        "max_time": np.max(times),
        "median_time": np.median(times),
        "memory_delta": memory_after - memory_before
    }

# Define test inputs for benchmarking
test_inputs = {
    "question_answering": {
        "question": "What is artificial intelligence?",
        "context": "Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of intelligent agents: any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals."
    },
    "summarization": {
        "text": """
        Machine learning is a method of data analysis that automates analytical model building. 
        It is a branch of artificial intelligence based on the idea that systems can learn from data, 
        identify patterns and make decisions with minimal human intervention. Machine learning algorithms 
        build mathematical models based on training data in order to make predictions or decisions without 
        being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of 
        applications, such as in medicine, email filtering, speech recognition, and computer vision, 
        where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.
        """
    }
}

# Run benchmarks
benchmark_results = {}

print("🔄 Running performance benchmarks...")
print("=" * 60)

# Benchmark original models
if ai_models:
    for task, model in ai_models.items():
        if model is not None and task in test_inputs:
            print(f"\n📊 Benchmarking original {task} model...")
            
            test_input = test_inputs[task]
            result = benchmark_model(model, test_input, f"Original {task}")
            benchmark_results[f"original_{task}"] = result
            
            if "error" not in result:
                print(f"  ⏱️  Mean time: {result['mean_time']:.3f}s ± {result['std_time']:.3f}s")
                print(f"  🚀 Best time: {result['min_time']:.3f}s")
                print(f"  💾 Memory delta: {result['memory_delta']:.1f} MB")
            else:
                print(f"  ❌ {result['error']}")

# Benchmark OpenVINO models
if openvino_models:
    for task, model in openvino_models.items():
        if model is not None:
            print(f"\n📊 Benchmarking OpenVINO {task} model...")
            
            # Map task names to test inputs
            test_key = "question_answering" if task == "qa" else task
            if test_key in test_inputs:
                test_input = test_inputs[test_key]
                result = benchmark_model(model, test_input, f"OpenVINO {task}")
                benchmark_results[f"openvino_{task}"] = result
                
                if "error" not in result:
                    print(f"  ⏱️  Mean time: {result['mean_time']:.3f}s ± {result['std_time']:.3f}s")
                    print(f"  🚀 Best time: {result['min_time']:.3f}s")
                    print(f"  💾 Memory delta: {result['memory_delta']:.1f} MB")
                else:
                    print(f"  ❌ {result['error']}")

print("\n" + "=" * 60)
print("✅ Benchmarking completed!")

# Create performance comparison DataFrame
performance_data = []
for key, result in benchmark_results.items():
    if "error" not in result:
        performance_data.append({
            "Model": result["model_name"],
            "Mean Time (s)": result["mean_time"],
            "Min Time (s)": result["min_time"],
            "Max Time (s)": result["max_time"],
            "Std Dev (s)": result["std_time"],
            "Memory Delta (MB)": result["memory_delta"]
        })

if performance_data:
    performance_df = pd.DataFrame(performance_data)
    print("\n📈 Performance Summary:")
    print(performance_df.round(3))

In [ ]:
# Visualize performance results
if performance_data:
    # Create performance comparison plots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Mean inference time comparison
    models = performance_df["Model"]
    mean_times = performance_df["Mean Time (s)"]
    
    ax1.bar(models, mean_times, color=['skyblue', 'lightcoral', 'lightgreen', 'orange'])
    ax1.set_title("Mean Inference Time Comparison")
    ax1.set_ylabel("Time (seconds)")
    ax1.tick_params(axis='x', rotation=45)
    
    # Add performance threshold line
    ax1.axhline(y=2.0, color='red', linestyle='--', alpha=0.7, label='Target: 2s')
    ax1.legend()
    
    # Min vs Max time comparison
    min_times = performance_df["Min Time (s)"]
    max_times = performance_df["Max Time (s)"]
    
    x = np.arange(len(models))
    width = 0.35
    
    ax2.bar(x - width/2, min_times, width, label='Min Time', color='lightblue')
    ax2.bar(x + width/2, max_times, width, label='Max Time', color='lightcoral')
    ax2.set_title("Min vs Max Inference Time")
    ax2.set_ylabel("Time (seconds)")
    ax2.set_xticks(x)
    ax2.set_xticklabels(models, rotation=45)
    ax2.legend()
    
    # Memory usage comparison
    memory_delta = performance_df["Memory Delta (MB)"]
    
    ax3.bar(models, memory_delta, color=['purple', 'green', 'orange', 'red'])
    ax3.set_title("Memory Usage Delta")
    ax3.set_ylabel("Memory (MB)")
    ax3.tick_params(axis='x', rotation=45)
    
    # Standard deviation (consistency)
    std_devs = performance_df["Std Dev (s)"]
    
    ax4.bar(models, std_devs, color=['gold', 'silver', 'bronze', 'copper'])
    ax4.set_title("Performance Consistency (Lower is Better)")
    ax4.set_ylabel("Standard Deviation (s)")
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Create interactive Plotly chart
    fig_plotly = go.Figure()
    
    # Add bars for each model
    for i, model in enumerate(models):
        fig_plotly.add_trace(go.Bar(
            name=model,
            x=[model],
            y=[mean_times.iloc[i]],
            error_y=dict(
                type='data',
                array=[std_devs.iloc[i]],
                visible=True
            ),
            text=f"{mean_times.iloc[i]:.3f}s",
            textposition="auto"
        ))
    
    # Add target line
    fig_plotly.add_hline(
        y=2.0, 
        line_dash="dash", 
        line_color="red",
        annotation_text="Target: 2s"
    )
    
    fig_plotly.update_layout(
        title="AI Model Performance Benchmark",
        xaxis_title="Model",
        yaxis_title="Inference Time (seconds)",
        showlegend=False,
        height=500
    )
    
    fig_plotly.show()

else:
    print("⚠️  No performance data available for visualization")

# Calculate speedup if both original and OpenVINO results exist
speedup_analysis = {}
for task in ["qa", "summarization"]:
    original_key = f"original_question_answering" if task == "qa" else f"original_{task}"
    openvino_key = f"openvino_{task}"
    
    if (original_key in benchmark_results and openvino_key in benchmark_results and
        "error" not in benchmark_results[original_key] and "error" not in benchmark_results[openvino_key]):
        
        original_time = benchmark_results[original_key]["mean_time"]
        openvino_time = benchmark_results[openvino_key]["mean_time"]
        speedup = original_time / openvino_time
        
        speedup_analysis[task] = {
            "original_time": original_time,
            "openvino_time": openvino_time,
            "speedup": speedup,
            "improvement": (original_time - openvino_time) / original_time * 100
        }

if speedup_analysis:
    print("\n🚀 OpenVINO Speedup Analysis:")
    print("=" * 50)
    for task, analysis in speedup_analysis.items():
        print(f"\n{task.upper()}:")
        print(f"  Original: {analysis['original_time']:.3f}s")
        print(f"  OpenVINO: {analysis['openvino_time']:.3f}s")
        print(f"  Speedup: {analysis['speedup']:.2f}x")
        print(f"  Improvement: {analysis['improvement']:.1f}%")

## 6. Build the Interactive Assistant Interface

### 🎨 User Interface Design

Our learning assistant interface supports multiple interaction modes:

1. **Text-based Q&A**: Traditional chat interface for questions
2. **Document Upload**: PDF/text file processing and summarization  
3. **Voice Interaction**: Speech-to-text and text-to-speech
4. **Multimodal**: Combined text, voice, and visual inputs

### 🛠️ Technology Choices

- **Gradio**: For rapid prototyping and demos
- **Streamlit**: For full-featured web applications
- **FastAPI**: For robust backend API services
- **WebRTC**: For real-time audio/video processing

In [ ]:
# Build interactive interface with Gradio
try:
    import gradio as gr
    gradio_available = True
    print("✅ Gradio is available")
except ImportError:
    print("📦 Installing Gradio...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio"])
    import gradio as gr
    gradio_available = True

# Define interface functions
def process_question(question: str, context: str = "") -> str:
    """Process a question using the QA model"""
    if not question.strip():
        return "Please enter a question."
    
    # Use OpenVINO model if available, otherwise fall back to original
    model = openvino_models.get("qa") or ai_models.get("question_answering")
    
    if model is None:
        return "❌ Question answering model not available."
    
    try:
        if context.strip():
            full_context = context
        else:
            # Provide default educational context
            full_context = "This is an educational question in a classroom setting."
        
        start_time = time.time()
        
        if hasattr(model, 'predict'):  # OpenVINO model
            result = model(question=question, context=full_context)
        else:  # HuggingFace pipeline
            result = model(question=question, context=full_context)
        
        processing_time = time.time() - start_time
        
        answer = result.get('answer', str(result))
        confidence = result.get('score', 0.0)
        
        response = f"**Answer:** {answer}\n\n"
        response += f"**Confidence:** {confidence:.2f}\n"
        response += f"**Processing Time:** {processing_time:.3f} seconds"
        
        return response
        
    except Exception as e:
        return f"❌ Error processing question: {str(e)}"

def summarize_text(text: str, max_length: int = 150) -> str:
    """Summarize text using the summarization model"""
    if not text.strip():
        return "Please enter text to summarize."
    
    if len(text) < 50:
        return "Text is too short to summarize effectively."
    
    # Use OpenVINO model if available, otherwise fall back to original
    model = openvino_models.get("summarization") or ai_models.get("summarization")
    
    if model is None:
        return "❌ Summarization model not available."
    
    try:
        start_time = time.time()
        
        if hasattr(model, 'generate'):  # OpenVINO model
            result = model(text, max_length=max_length, min_length=30, do_sample=False)
        else:  # HuggingFace pipeline
            result = model(text, max_length=max_length, min_length=30, do_sample=False)
        
        processing_time = time.time() - start_time
        
        if isinstance(result, list) and len(result) > 0:
            summary = result[0].get('summary_text', str(result[0]))
        else:
            summary = str(result)
        
        response = f"**Summary:** {summary}\n\n"
        response += f"**Original Length:** {len(text)} characters\n"
        response += f"**Summary Length:** {len(summary)} characters\n"
        response += f"**Compression Ratio:** {len(summary)/len(text):.2%}\n"
        response += f"**Processing Time:** {processing_time:.3f} seconds"
        
        return response
        
    except Exception as e:
        return f"❌ Error summarizing text: {str(e)}"

def analyze_learning_content(content: str) -> str:
    """Analyze educational content and provide insights"""
    if not content.strip():
        return "Please provide content to analyze."
    
    # Basic educational content analysis
    words = content.split()
    sentences = content.split('.')
    
    # Calculate readability metrics
    avg_words_per_sentence = len(words) / len(sentences) if sentences else 0
    avg_word_length = sum(len(word) for word in words) / len(words) if words else 0
    
    # Determine difficulty level
    if avg_word_length < 5 and avg_words_per_sentence < 15:
        difficulty = "Beginner"
    elif avg_word_length < 6 and avg_words_per_sentence < 20:
        difficulty = "Intermediate"
    else:
        difficulty = "Advanced"
    
    # Estimate reading time (200 WPM average)
    reading_time = len(words) / 200
    
    # Generate summary
    summary = summarize_text(content)
    
    analysis = f"""
**📊 Content Analysis:**

**📈 Metrics:**
- Word Count: {len(words)}
- Sentence Count: {len(sentences)}
- Average Words per Sentence: {avg_words_per_sentence:.1f}
- Average Word Length: {avg_word_length:.1f} characters
- Estimated Reading Time: {reading_time:.1f} minutes

**🎯 Difficulty Level:** {difficulty}

**📝 Summary:**
{summary}

**💡 Teaching Suggestions:**
- Break down into {max(1, len(words)//100)} smaller sections
- Add visual aids for complex concepts
- Include interactive elements every {max(1, int(reading_time))} minutes
    """
    
    return analysis

# Create Gradio interface
def create_learning_assistant_interface():
    """Create the main Gradio interface"""
    
    with gr.Blocks(title="🎓 AI Learning Assistant", theme=gr.themes.Soft()) as interface:
        
        gr.Markdown("""
        # 🎓 AI-Powered Interactive Learning Assistant
        
        Welcome to your intelligent classroom companion! This assistant helps with:
        - ❓ **Question Answering**: Get instant answers to your questions
        - 📝 **Content Summarization**: Summarize lessons and materials  
        - 🔍 **Learning Analysis**: Analyze content difficulty and provide teaching tips
        
        *Powered by OpenVINO for optimal performance*
        """)
        
        with gr.Tab("❓ Question & Answer"):
            gr.Markdown("### Ask any educational question and get instant answers!")
            
            with gr.Row():
                with gr.Column():
                    question_input = gr.Textbox(
                        label="Your Question",
                        placeholder="What is photosynthesis?",
                        lines=2
                    )
                    context_input = gr.Textbox(
                        label="Context (optional)",
                        placeholder="Provide additional context or lesson material...",
                        lines=4
                    )
                    ask_btn = gr.Button("Ask Question", variant="primary")
                
                with gr.Column():
                    answer_output = gr.Markdown(label="Answer")
            
            ask_btn.click(
                fn=process_question,
                inputs=[question_input, context_input],
                outputs=answer_output
            )
            
            # Example questions
            gr.Examples(
                examples=[
                    ["What is machine learning?", "Machine learning is a subset of artificial intelligence."],
                    ["How does photosynthesis work?", "Photosynthesis is the process by which plants make food using sunlight."],
                    ["What caused World War II?", "World War II was a global conflict from 1939-1945."]
                ],
                inputs=[question_input, context_input]
            )
        
        with gr.Tab("📝 Text Summarization"):
            gr.Markdown("### Summarize any educational content quickly!")
            
            with gr.Row():
                with gr.Column():
                    text_input = gr.Textbox(
                        label="Text to Summarize",
                        placeholder="Paste your lesson content, article, or any text here...",
                        lines=10
                    )
                    max_length_slider = gr.Slider(
                        minimum=50,
                        maximum=300,
                        value=150,
                        step=10,
                        label="Summary Length (words)"
                    )
                    summarize_btn = gr.Button("Generate Summary", variant="primary")
                
                with gr.Column():
                    summary_output = gr.Markdown(label="Summary")
            
            summarize_btn.click(
                fn=summarize_text,
                inputs=[text_input, max_length_slider],
                outputs=summary_output
            )
        
        with gr.Tab("🔍 Learning Analysis"):
            gr.Markdown("### Analyze educational content and get teaching insights!")
            
            with gr.Row():
                with gr.Column():
                    content_input = gr.Textbox(
                        label="Educational Content",
                        placeholder="Paste lesson content for analysis...",
                        lines=12
                    )
                    analyze_btn = gr.Button("Analyze Content", variant="primary")
                
                with gr.Column():
                    analysis_output = gr.Markdown(label="Analysis Results")
            
            analyze_btn.click(
                fn=analyze_learning_content,
                inputs=content_input,
                outputs=analysis_output
            )
        
        with gr.Tab("📊 Performance Monitor"):
            gr.Markdown("### System Performance and Model Information")
            
            # Display model status
            model_status = ""
            for task, model in ai_models.items():
                status = "✅ Loaded" if model else "❌ Not Available"
                model_status += f"- **{task.title()}**: {status}\n"
            
            if openvino_models:
                model_status += "\n**OpenVINO Models:**\n"
                for task, model in openvino_models.items():
                    status = "✅ Optimized" if model else "❌ Not Available" 
                    model_status += f"- **{task.title()}**: {status}\n"
            
            gr.Markdown(f"### Model Status\n{model_status}")
            
            # Display performance data if available
            if benchmark_results:
                perf_text = "### Performance Benchmarks\n\n"
                for model_name, results in benchmark_results.items():
                    if "error" not in results:
                        perf_text += f"**{results['model_name']}:**\n"
                        perf_text += f"- Mean Time: {results['mean_time']:.3f}s\n"
                        perf_text += f"- Best Time: {results['min_time']:.3f}s\n\n"
                
                gr.Markdown(perf_text)
    
    return interface

# Launch the interface
if gradio_available:
    print("\n🚀 Creating Gradio interface...")
    demo = create_learning_assistant_interface()
    
    print("✅ Interface created successfully!")
    print("💡 Run demo.launch() to start the interactive interface")
    
    # To launch immediately (uncomment the line below):
    # demo.launch(share=True, debug=True)
else:
    print("❌ Could not create interface - Gradio not available")

## 7. Integrate Backend Logic for Multimodal Interaction

### 🔀 Input Routing Strategy

Our system intelligently routes different input types to appropriate AI models:

```mermaid
graph TD
    A[User Input] --> B{Input Type?}
    B -->|Text Question| C[Question Answering Model]
    B -->|Long Text| D[Summarization Model]
    B -->|Audio File| E[Speech Recognition Model]
    B -->|Image| F[Image Captioning Model]
    C --> G[Educational Enhancement]
    D --> G
    E --> G
    F --> G
    G --> H[Response Generation]
    H --> I[Output Formatting]
```

### 🎯 Multimodal Processing Pipeline

1. **Input Detection**: Identify content type and format
2. **Preprocessing**: Clean and prepare data for models
3. **Model Selection**: Route to appropriate AI model
4. **Processing**: Generate initial response
5. **Enhancement**: Add educational features
6. **Output**: Format for user consumption

In [ ]:
# Implement multimodal processing pipeline
from enum import Enum
from dataclasses import dataclass
from typing import Union, Any, Optional

class InputType(Enum):
    TEXT_QUESTION = "text_question"
    TEXT_DOCUMENT = "text_document"
    AUDIO_FILE = "audio_file"
    IMAGE_FILE = "image_file"
    MULTIMODAL = "multimodal"

@dataclass
class ProcessingResult:
    """Structured result from processing pipeline"""
    input_type: InputType
    original_input: Any
    processed_output: str
    confidence: float
    processing_time: float
    educational_enhancements: dict
    metadata: dict

class MultimodalProcessor:
    """Main processor for handling different input types"""
    
    def __init__(self, models_dict: dict, openvino_models_dict: dict = None):
        self.models = models_dict
        self.openvino_models = openvino_models_dict or {}
        
    def detect_input_type(self, input_data: Any) -> InputType:
        """Detect the type of input provided"""
        
        if isinstance(input_data, str):
            # Analyze text to determine if it's a question or document
            text = input_data.strip().lower()
            
            # Check for question indicators
            question_words = ["what", "why", "how", "when", "where", "who", "can", "could", "would", "should"]
            is_question = (
                text.endswith("?") or
                any(text.startswith(word) for word in question_words) or
                len(text.split()) < 20  # Short text likely a question
            )
            
            return InputType.TEXT_QUESTION if is_question else InputType.TEXT_DOCUMENT
            
        elif hasattr(input_data, 'name') and input_data.name:
            # File-like object
            filename = input_data.name.lower()
            if filename.endswith(('.mp3', '.wav', '.m4a', '.ogg')):
                return InputType.AUDIO_FILE
            elif filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                return InputType.IMAGE_FILE
                
        return InputType.TEXT_QUESTION  # Default fallback
    
    def get_best_model(self, task: str):
        """Get the best available model for a task (OpenVINO preferred)"""
        # Try OpenVINO first, then fall back to original
        model_mapping = {
            "question_answering": ["qa", "question_answering"],
            "summarization": ["summarization", "summarization"],
            "text_generation": ["text_generation", "text_generation"]
        }
        
        possible_keys = model_mapping.get(task, [task])
        
        # Check OpenVINO models first
        for key in possible_keys:
            if key in self.openvino_models and self.openvino_models[key] is not None:
                return self.openvino_models[key], "openvino"
        
        # Fall back to original models
        for key in possible_keys:
            if key in self.models and self.models[key] is not None:
                return self.models[key], "original"
        
        return None, None
    
    def process_text_question(self, question: str, context: str = "") -> ProcessingResult:
        """Process a text question"""
        start_time = time.time()
        
        model, model_type = self.get_best_model("question_answering")
        
        if model is None:
            return ProcessingResult(
                input_type=InputType.TEXT_QUESTION,
                original_input=question,
                processed_output="❌ Question answering model not available",
                confidence=0.0,
                processing_time=0.0,
                educational_enhancements={},
                metadata={"error": "Model not available"}
            )
        
        try:
            # Provide educational context if none given
            if not context.strip():
                context = "This is an educational question in a classroom setting. Please provide a clear, educational answer suitable for learning."
            
            # Process with model
            result = model(question=question, context=context)
            processing_time = time.time() - start_time
            
            # Extract answer and confidence
            if isinstance(result, dict):
                answer = result.get('answer', str(result))
                confidence = result.get('score', 0.0)
            else:
                answer = str(result)
                confidence = 0.8  # Default confidence
            
            # Generate educational enhancements
            enhancements = self.generate_educational_enhancements(question, answer)
            
            return ProcessingResult(
                input_type=InputType.TEXT_QUESTION,
                original_input=question,
                processed_output=answer,
                confidence=confidence,
                processing_time=processing_time,
                educational_enhancements=enhancements,
                metadata={"model_type": model_type, "context_provided": bool(context.strip())}
            )
            
        except Exception as e:
            return ProcessingResult(
                input_type=InputType.TEXT_QUESTION,
                original_input=question,
                processed_output=f"❌ Error processing question: {str(e)}",
                confidence=0.0,
                processing_time=time.time() - start_time,
                educational_enhancements={},
                metadata={"error": str(e)}
            )
    
    def process_text_document(self, text: str, max_length: int = 150) -> ProcessingResult:
        """Process a text document for summarization"""
        start_time = time.time()
        
        model, model_type = self.get_best_model("summarization")
        
        if model is None:
            return ProcessingResult(
                input_type=InputType.TEXT_DOCUMENT,
                original_input=text,
                processed_output="❌ Summarization model not available",
                confidence=0.0,
                processing_time=0.0,
                educational_enhancements={},
                metadata={"error": "Model not available"}
            )
        
        try:
            # Process with model
            result = model(text, max_length=max_length, min_length=30, do_sample=False)
            processing_time = time.time() - start_time
            
            # Extract summary
            if isinstance(result, list) and len(result) > 0:
                summary = result[0].get('summary_text', str(result[0]))
            else:
                summary = str(result)
            
            # Calculate metrics
            compression_ratio = len(summary) / len(text) if len(text) > 0 else 0
            confidence = min(1.0, max(0.0, 1.0 - compression_ratio))  # Higher compression = lower confidence
            
            # Generate educational enhancements
            enhancements = {
                "original_length": len(text),
                "summary_length": len(summary),
                "compression_ratio": compression_ratio,
                "reading_time_original": len(text.split()) / 200,  # 200 WPM
                "reading_time_summary": len(summary.split()) / 200,
                "key_topics": self.extract_key_topics(text),
                "difficulty_level": self.assess_difficulty(text)
            }
            
            return ProcessingResult(
                input_type=InputType.TEXT_DOCUMENT,
                original_input=text,
                processed_output=summary,
                confidence=confidence,
                processing_time=processing_time,
                educational_enhancements=enhancements,
                metadata={"model_type": model_type}
            )
            
        except Exception as e:
            return ProcessingResult(
                input_type=InputType.TEXT_DOCUMENT,
                original_input=text,
                processed_output=f"❌ Error summarizing text: {str(e)}",
                confidence=0.0,
                processing_time=time.time() - start_time,
                educational_enhancements={},
                metadata={"error": str(e)}
            )
    
    def generate_educational_enhancements(self, question: str, answer: str) -> dict:
        """Generate educational enhancements for Q&A"""
        enhancements = {}
        
        # Generate follow-up questions
        question_lower = question.lower()
        if "what" in question_lower:
            enhancements["follow_up_questions"] = [
                "Why is this important?",
                "How does this relate to other concepts?",
                "Can you provide an example?"
            ]
        elif "why" in question_lower:
            enhancements["follow_up_questions"] = [
                "What are the implications?",
                "How could this be different?",
                "What evidence supports this?"
            ]
        elif "how" in question_lower:
            enhancements["follow_up_questions"] = [
                "What are the key steps?",
                "Why does this method work?",
                "Are there alternative approaches?"
            ]
        else:
            enhancements["follow_up_questions"] = [
                "What would you like to know more about?",
                "How can this be applied?",
                "What are related concepts?"
            ]
        
        # Generate learning tips
        enhancements["learning_tips"] = [
            "Try to connect this to what you already know",
            "Practice explaining this concept in your own words",
            "Look for real-world examples of this concept"
        ]
        
        # Assess complexity
        complexity_score = (len(answer.split()) / 50) + (len([w for w in answer.split() if len(w) > 6]) / 10)
        if complexity_score < 1:
            complexity = "Simple"
        elif complexity_score < 2:
            complexity = "Moderate"
        else:
            complexity = "Complex"
        
        enhancements["complexity"] = complexity
        enhancements["estimated_read_time"] = len(answer.split()) / 200  # minutes
        
        return enhancements
    
    def extract_key_topics(self, text: str) -> list:
        """Extract key topics from text (simplified approach)"""
        # Simple keyword extraction - in production, use more sophisticated NLP
        words = text.lower().split()
        
        # Common educational topic indicators
        topic_indicators = [
            "mathematics", "science", "history", "literature", "biology", 
            "chemistry", "physics", "geography", "economics", "psychology"
        ]
        
        found_topics = [topic for topic in topic_indicators if topic in text.lower()]
        
        # Also extract frequently occurring longer words
        long_words = [word for word in words if len(word) > 6]
        word_freq = {}
        for word in long_words:
            word_freq[word] = word_freq.get(word, 0) + 1
        
        # Get top frequent words
        frequent_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:5]
        found_topics.extend([word for word, freq in frequent_words if freq > 1])
        
        return found_topics[:10]  # Limit to top 10
    
    def assess_difficulty(self, text: str) -> str:
        """Assess text difficulty level"""
        words = text.split()
        sentences = text.split('.')
        
        avg_word_length = sum(len(word) for word in words) / len(words) if words else 0
        avg_sentence_length = len(words) / len(sentences) if sentences else 0
        
        if avg_word_length < 5 and avg_sentence_length < 15:
            return "Beginner"
        elif avg_word_length < 6 and avg_sentence_length < 20:
            return "Intermediate"
        else:
            return "Advanced"
    
    def process(self, input_data: Any, **kwargs) -> ProcessingResult:
        """Main processing method that routes to appropriate handler"""
        
        input_type = self.detect_input_type(input_data)
        
        if input_type == InputType.TEXT_QUESTION:
            context = kwargs.get('context', '')
            return self.process_text_question(input_data, context)
            
        elif input_type == InputType.TEXT_DOCUMENT:
            max_length = kwargs.get('max_length', 150)
            return self.process_text_document(input_data, max_length)
            
        elif input_type == InputType.AUDIO_FILE:
            # Placeholder for audio processing
            return ProcessingResult(
                input_type=InputType.AUDIO_FILE,
                original_input=input_data,
                processed_output="🎤 Audio processing not implemented in this demo",
                confidence=0.0,
                processing_time=0.0,
                educational_enhancements={},
                metadata={"note": "Would use speech recognition model"}
            )
            
        elif input_type == InputType.IMAGE_FILE:
            # Placeholder for image processing
            return ProcessingResult(
                input_type=InputType.IMAGE_FILE,
                original_input=input_data,
                processed_output="🖼️ Image processing not implemented in this demo",
                confidence=0.0,
                processing_time=0.0,
                educational_enhancements={},
                metadata={"note": "Would use image captioning model"}
            )
        
        else:
            return ProcessingResult(
                input_type=input_type,
                original_input=input_data,
                processed_output="❌ Unsupported input type",
                confidence=0.0,
                processing_time=0.0,
                educational_enhancements={},
                metadata={"error": "Unsupported input"}
            )

# Initialize the multimodal processor
processor = MultimodalProcessor(ai_models, openvino_models)

print("✅ Multimodal processor initialized!")
print(f"📊 Available models: {list(ai_models.keys())}")
if openvino_models:
    print(f"🚀 OpenVINO models: {list(openvino_models.keys())}")

## 8. Implement Personalization and Progress Tracking

### 🎯 Personalization Strategy

Our learning assistant adapts to individual students through:

1. **Learning Style Detection**: Analyze preferred interaction modes
2. **Difficulty Adaptation**: Adjust response complexity based on comprehension
3. **Topic Tracking**: Monitor subject areas and knowledge gaps
4. **Performance Analytics**: Track response times and accuracy trends

### 📊 Progress Tracking Features

- **Session History**: Store questions, answers, and interactions
- **Learning Patterns**: Identify strengths and improvement areas
- **Engagement Metrics**: Monitor interaction frequency and depth
- **Adaptive Responses**: Customize explanations based on student level

In [ ]:
# Implement personalization and progress tracking
from datetime import datetime, timedelta
from collections import defaultdict, Counter
import json

@dataclass
class StudentProfile:
    """Student learning profile and preferences"""
    student_id: str
    name: str = ""
    grade_level: str = ""
    learning_style: str = "visual"  # visual, auditory, kinesthetic
    preferred_complexity: str = "intermediate"  # beginner, intermediate, advanced
    strong_subjects: list = None
    weak_subjects: list = None
    interaction_history: list = None
    performance_metrics: dict = None
    
    def __post_init__(self):
        if self.strong_subjects is None:
            self.strong_subjects = []
        if self.weak_subjects is None:
            self.weak_subjects = []
        if self.interaction_history is None:
            self.interaction_history = []
        if self.performance_metrics is None:
            self.performance_metrics = {
                "total_questions": 0,
                "correct_answers": 0,
                "avg_response_time": 0.0,
                "engagement_score": 0.0,
                "last_activity": None
            }

class LearningAnalytics:
    """Analyze learning patterns and provide insights"""
    
    def __init__(self):
        self.students = {}  # student_id -> StudentProfile
        self.session_data = defaultdict(list)  # session_id -> interactions
    
    def get_or_create_student(self, student_id: str, **kwargs) -> StudentProfile:
        """Get existing student profile or create new one"""
        if student_id not in self.students:
            self.students[student_id] = StudentProfile(student_id=student_id, **kwargs)
        return self.students[student_id]
    
    def record_interaction(self, student_id: str, interaction_data: dict):
        """Record a learning interaction"""
        student = self.get_or_create_student(student_id)
        
        # Add timestamp
        interaction_data["timestamp"] = datetime.now().isoformat()
        
        # Store in student history
        student.interaction_history.append(interaction_data)
        
        # Update performance metrics
        self.update_performance_metrics(student, interaction_data)
        
        # Limit history size
        if len(student.interaction_history) > 100:
            student.interaction_history = student.interaction_history[-100:]
    
    def update_performance_metrics(self, student: StudentProfile, interaction: dict):
        """Update student performance metrics"""
        metrics = student.performance_metrics
        
        # Update total questions
        if interaction.get("type") == "question":
            metrics["total_questions"] += 1
            
            # Update response time
            response_time = interaction.get("processing_time", 0)
            if metrics["avg_response_time"] == 0:
                metrics["avg_response_time"] = response_time
            else:
                # Running average
                total_questions = metrics["total_questions"]
                metrics["avg_response_time"] = (
                    (metrics["avg_response_time"] * (total_questions - 1) + response_time) / total_questions
                )
            
            # Update engagement score based on interaction quality
            confidence = interaction.get("confidence", 0.5)
            metrics["engagement_score"] = (
                metrics["engagement_score"] * 0.9 + confidence * 0.1
            )
        
        # Update last activity
        metrics["last_activity"] = datetime.now().isoformat()
    
    def analyze_learning_style(self, student_id: str) -> dict:
        """Analyze student's learning style preferences"""
        student = self.students.get(student_id)
        if not student or not student.interaction_history:
            return {"style": "visual", "confidence": 0.0}
        
        # Analyze interaction patterns
        interaction_types = Counter()
        success_by_type = defaultdict(list)
        
        for interaction in student.interaction_history[-20:]:  # Last 20 interactions
            interaction_type = interaction.get("input_type", "text")
            confidence = interaction.get("confidence", 0.5)
            
            interaction_types[interaction_type] += 1
            success_by_type[interaction_type].append(confidence)
        
        # Determine preferred style
        style_mapping = {
            "text_question": "visual",
            "text_document": "visual", 
            "audio_file": "auditory",
            "image_file": "visual"
        }
        
        style_scores = defaultdict(float)
        for itype, count in interaction_types.items():
            style = style_mapping.get(itype, "visual")
            avg_success = np.mean(success_by_type[itype]) if success_by_type[itype] else 0.5
            style_scores[style] += count * avg_success
        
        if style_scores:
            best_style = max(style_scores.items(), key=lambda x: x[1])
            total_score = sum(style_scores.values())
            confidence = best_style[1] / total_score if total_score > 0 else 0.0
            
            return {"style": best_style[0], "confidence": confidence}
        
        return {"style": "visual", "confidence": 0.0}
    
    def assess_knowledge_level(self, student_id: str, subject: str = None) -> dict:
        """Assess student's knowledge level in a subject"""
        student = self.students.get(student_id)
        if not student or not student.interaction_history:
            return {"level": "beginner", "confidence": 0.0}
        
        # Filter interactions by subject if specified
        relevant_interactions = student.interaction_history
        if subject:
            relevant_interactions = [
                i for i in student.interaction_history 
                if subject.lower() in i.get("original_input", "").lower()
            ]
        
        if not relevant_interactions:
            return {"level": "beginner", "confidence": 0.0}
        
        # Analyze complexity of questions asked and confidence of answers
        complexities = []
        confidences = []
        
        for interaction in relevant_interactions[-10:]:  # Last 10 relevant interactions
            # Get complexity from educational enhancements
            enhancements = interaction.get("educational_enhancements", {})
            complexity = enhancements.get("complexity", "Simple")
            
            complexity_score = {"Simple": 1, "Moderate": 2, "Complex": 3}.get(complexity, 1)
            complexities.append(complexity_score)
            
            confidence = interaction.get("confidence", 0.5)
            confidences.append(confidence)
        
        if complexities and confidences:
            avg_complexity = np.mean(complexities)
            avg_confidence = np.mean(confidences)
            
            # Determine level based on complexity handled and confidence
            level_score = avg_complexity * avg_confidence
            
            if level_score < 1.5:
                level = "beginner"
            elif level_score < 2.5:
                level = "intermediate"  
            else:
                level = "advanced"
            
            return {"level": level, "confidence": avg_confidence}
        
        return {"level": "beginner", "confidence": 0.0}
    
    def generate_personalized_response(self, student_id: str, base_response: str, 
                                     interaction_type: str) -> str:
        """Generate personalized response based on student profile"""
        student = self.students.get(student_id)
        if not student:
            return base_response
        
        # Analyze student's learning style and level
        style_analysis = self.analyze_learning_style(student_id)
        learning_style = style_analysis["style"]
        
        # Get performance metrics
        metrics = student.performance_metrics
        engagement = metrics.get("engagement_score", 0.5)
        
        # Customize response based on learning style
        if learning_style == "visual":
            suggestion = "\n\n💡 **Visual Learning Tip**: Try creating a diagram or mind map of this concept."
        elif learning_style == "auditory":
            suggestion = "\n\n🎵 **Auditory Learning Tip**: Try explaining this concept out loud or discussing it with others."
        else:  # kinesthetic
            suggestion = "\n\n🤲 **Hands-on Learning Tip**: Look for ways to practice or apply this concept."
        
        # Add encouragement based on engagement
        if engagement < 0.4:
            encouragement = "\n\n✨ **Keep going!** Learning takes time, and you're making progress."
        elif engagement > 0.8:
            encouragement = "\n\n🌟 **Great job!** You're showing excellent understanding."
        else:
            encouragement = "\n\n👍 **Nice work!** You're building good knowledge."
        
        return base_response + suggestion + encouragement
    
    def get_learning_dashboard(self, student_id: str) -> dict:
        """Generate learning dashboard for student"""
        student = self.students.get(student_id)
        if not student:
            return {"error": "Student not found"}
        
        # Recent activity
        recent_interactions = student.interaction_history[-10:] if student.interaction_history else []
        
        # Performance trends
        metrics = student.performance_metrics
        
        # Learning style analysis
        style_analysis = self.analyze_learning_style(student_id)
        
        # Subject analysis
        subject_performance = {}
        subjects = ["mathematics", "science", "history", "english", "geography"]
        for subject in subjects:
            assessment = self.assess_knowledge_level(student_id, subject)
            subject_performance[subject] = assessment
        
        return {
            "student_profile": {
                "name": student.name or f"Student {student_id}",
                "grade_level": student.grade_level,
                "learning_style": style_analysis,
                "total_interactions": len(student.interaction_history)
            },
            "performance_metrics": metrics,
            "subject_performance": subject_performance,
            "recent_activity": recent_interactions[-5:],  # Last 5 interactions
            "recommendations": self.generate_recommendations(student_id)
        }
    
    def generate_recommendations(self, student_id: str) -> list:
        """Generate personalized learning recommendations"""
        student = self.students.get(student_id)
        if not student or not student.interaction_history:
            return ["Start by asking questions about topics you're curious about!"]
        
        recommendations = []
        
        # Analyze recent activity
        recent = student.interaction_history[-5:] if student.interaction_history else []
        if not recent:
            return ["Continue exploring different topics to build your knowledge."]
        
        # Check for engagement patterns
        recent_confidences = [i.get("confidence", 0.5) for i in recent]
        avg_confidence = np.mean(recent_confidences)
        
        if avg_confidence < 0.4:
            recommendations.append("💪 Try asking simpler questions to build confidence")
            recommendations.append("📚 Review basic concepts before tackling complex topics")
        elif avg_confidence > 0.8:
            recommendations.append("🚀 Challenge yourself with more advanced questions")
            recommendations.append("🔗 Explore connections between different topics")
        
        # Check interaction diversity
        interaction_types = [i.get("input_type", "text") for i in recent]
        if len(set(interaction_types)) <= 1:
            recommendations.append("🎯 Try different ways of learning (text, audio, visual)")
        
        # Subject-specific recommendations
        subjects_asked = []
        for interaction in recent:
            text = interaction.get("original_input", "").lower()
            for subject in ["math", "science", "history", "english"]:
                if subject in text:
                    subjects_asked.append(subject)
        
        if not subjects_asked:
            recommendations.append("🌍 Explore different subjects to find your interests")
        elif len(set(subjects_asked)) <= 1:
            recommendations.append("📖 Try asking questions about different subjects")
        
        return recommendations[:3]  # Limit to 3 recommendations

# Initialize learning analytics
analytics = LearningAnalytics()

# Demonstrate personalization
def demo_personalization():
    """Demonstrate personalization features"""
    print("🎯 Personalization Demo")
    print("=" * 40)
    
    # Create sample student
    student_id = "demo_student_001"
    
    # Simulate some interactions
    sample_interactions = [
        {
            "type": "question",
            "input_type": "text_question",
            "original_input": "What is photosynthesis?",
            "processed_output": "Photosynthesis is the process by which plants convert sunlight into energy.",
            "confidence": 0.85,
            "processing_time": 1.2,
            "educational_enhancements": {"complexity": "Simple"}
        },
        {
            "type": "question", 
            "input_type": "text_question",
            "original_input": "How does quantum mechanics relate to chemistry?",
            "processed_output": "Quantum mechanics explains atomic behavior that determines chemical bonding.",
            "confidence": 0.75,
            "processing_time": 2.1,
            "educational_enhancements": {"complexity": "Complex"}
        },
        {
            "type": "summarization",
            "input_type": "text_document", 
            "original_input": "Long text about World War II...",
            "processed_output": "Summary of WWII causes and effects.",
            "confidence": 0.9,
            "processing_time": 1.8,
            "educational_enhancements": {"complexity": "Moderate"}
        }
    ]
    
    # Record interactions
    for interaction in sample_interactions:
        analytics.record_interaction(student_id, interaction)
    
    # Generate dashboard
    dashboard = analytics.get_learning_dashboard(student_id)
    
    print("📊 Student Dashboard:")
    print(f"  Learning Style: {dashboard['student_profile']['learning_style']['style']}")
    print(f"  Total Interactions: {dashboard['student_profile']['total_interactions']}")
    print(f"  Engagement Score: {dashboard['performance_metrics']['engagement_score']:.2f}")
    
    print("\n🎯 Subject Performance:")
    for subject, performance in dashboard['subject_performance'].items():
        print(f"  {subject.title()}: {performance['level']} (confidence: {performance['confidence']:.2f})")
    
    print("\n💡 Recommendations:")
    for rec in dashboard['recommendations']:
        print(f"  • {rec}")
    
    # Demo personalized response
    base_response = "The water cycle involves evaporation, condensation, and precipitation."
    personalized = analytics.generate_personalized_response(
        student_id, base_response, "text_question"
    )
    
    print("\n🎨 Personalized Response Example:")
    print(f"Original: {base_response}")
    print(f"Personalized: {personalized}")

demo_personalization()

print("\n✅ Personalization and progress tracking system implemented!")

## 9. Demo: Real-Time Interaction and Performance Metrics

### 🎭 Live Demonstration

This section provides an interactive demonstration of our AI-powered learning assistant with real-time performance monitoring.

### 📊 Demo Features

1. **Question Answering**: Real-time educational Q&A
2. **Content Summarization**: Live text summarization  
3. **Multimodal Processing**: Handle different input types
4. **Performance Monitoring**: Live metrics and benchmarks
5. **Personalization**: Adaptive responses based on student profile

### 🎯 Performance Targets

- **Response Time**: < 2 seconds for questions
- **Accuracy**: > 90% confidence for educational content
- **Throughput**: Handle 10+ concurrent users
- **Memory Usage**: < 4GB RAM utilization